# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [4]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [6]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [4]:
def P1(num_examples=5):
### STUDENT START ###

    for i in range(num_examples):
        print "LABEL:",train_labels[i],"\nMESSAGE:",train_data[i],'\n' 
    
### STUDENT END ###

P1()

LABEL: 1 
MESSAGE: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych 

LABEL: 3 
MESSAGE: 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries. 

LABEL: 2 
MESSAGE: 

In [2]:
class ProblemAnswer:
    ''' Create an over-ambitious answer object '''
    def __init__(self, num_solutions):
        ''' Initalize answer object'''
        _alphabet = list('abcdefghijklmnopqrstuvwxyz')
        self.answers = {a:'' for a in _alphabet[:num_solutions]}
    
    def __str__(self):
        ''' Print all answers 
            to finish question 
        '''
        drop_ns = lambda x: (x.replace('\n','')).replace('  ','')
        answers = sorted([ 
            drop_ns(z+') '
            +self.answers[z]) for z in self.answers 
        ])
        return '\n\n'.join(answers)
    
    def add(self, answer_val, answer):
        ''' Add an answer'''
        self.answers[answer_val] = answer

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [6]:
def P2():
    
### STUDENT START ###

    ''' Initialize over-ambitious answer object 
    '''
    p2 = ProblemAnswer(6)
    
    ''' Create vectorizer and fit data into 
        feature vectors
    '''
    cv = CountVectorizer()
    X = cv.fit_transform(train_data)
    
    ''' Gather size and shape 
    '''
    _nonzero_values = X.nnz
    _vocab_size, _entries = X.shape[1], X.shape[0]*X.shape[1]
    a = '''
        The vocabulary size is {1}, meaning there are 
        {1} distinct words in the dataset. The number of 
        non-zero features is {0}, given by the 'nnz' 
        attribute of scipy's sparse coo_matrix. The percent 
        of entries that are nonzero is {2}% (or {0}/{3}).
    '''.format(
        _nonzero_values
        ,_vocab_size
        ,round(_nonzero_values*100.0 / _entries,2)
        ,_entries
    )
    p2.add('a',a)
    
    ''' Get first and last feature strings 
    '''
    _feature_names = cv.get_feature_names()
    _first,_last = _feature_names[:1], _feature_names[-1:]
    b = '''
        The first feature string is '{0}', and the last 
        feature string is '{1}'.
    '''.format(_first[0],_last[0])
    p2.add('b',b)
    
    ''' Specify vocabulary with only 4 distinct words '''
    Y = cv.fit_transform(["atheism", "graphics", "space", "religion"])
    _nonzero_values = Y.nnz
    _vocab_size, _entries = Y.shape[1], Y.shape[0]*Y.shape[1]
    c = '''
        The vocabulary size is {1}, meaning there are 
        {1} distinct words in the dataset. The number of 
        non-zero features is {0}, given by the 'nnz' 
        attribute of scipy's sparse coo_matrix. The percent 
        of entries that are nonzero is {2}% (or {0}/{3}).
    '''.format(
        _nonzero_values
        ,_vocab_size
        ,round(_nonzero_values*100.0 / _entries,2)
        ,_entries
    )
    p2.add('c',c)
    
    ''' Fit CV with bigram and trigram feature extraction '''
    cv = CountVectorizer(analyzer='char_wb',ngram_range = (2,3))
    Z = cv.fit_transform(train_data)
    _nonzero_values = Z.nnz
    _vocab_size, _entries = Z.shape[1], Z.shape[0]*Z.shape[1]
    d = '''
        The vocabulary size with bigram and trigram 
        extraction is {1}, meaning there are 
        {1} distinct words in the dataset. The number of 
        non-zero features is {0}. The percent 
        of entries that are nonzero is {2}% (or {0}/{3}).
    '''.format(
        _nonzero_values
        ,_vocab_size
        ,round(_nonzero_values*100.0 / _entries,2)
        ,_entries
    )
    p2.add('d',d)
    
    ''' Fit with specified min_df value of 10 '''
    cv = CountVectorizer(min_df=10)
    X10 = cv.fit_transform(train_data)
    _vocab_size = X10.shape[1]
    e = '''
        The vocabulary size with min_df set to exclude 
        values that show up in fewer than 10 documents 
        is {0}.
    '''.format(
        _vocab_size
    )
    p2.add('e',e)
    
    ''' Compare dev and train matrices '''
    cv_train, cv_dev = CountVectorizer(), CountVectorizer()
    X_train, X_dev = cv_train.fit_transform(train_data), cv_dev.fit_transform(dev_data)
    vocab_train, vocab_dev = X_train.shape[1], X_dev.shape[1]
    f = '''
        The vocabulary for the training set is {0}, while 
        the vocabulary for the dev set is {1}, which means 
        that the dev data set has {2}% of the distinct words 
        that the training dataset contains.
    '''.format(
        vocab_train
        ,vocab_dev
        ,round(vocab_dev*100.0 / vocab_train,2)
    )
    p2.add('f',f)

    print p2

### STUDENT END ###
P2()

a) The vocabulary size is 26879, meaning there are 26879 distinct words in the dataset. The number of non-zero features is 196700, given by the 'nnz' attribute of scipy's sparse coo_matrix. The percent of entries that are nonzero is 0.36% (or 196700/54671886).

b) The first feature string is '00', and the last feature string is 'zyxel'.

c) The vocabulary size is 4, meaning there are 4 distinct words in the dataset. The number of non-zero features is 4, given by the 'nnz' attribute of scipy's sparse coo_matrix. The percent of entries that are nonzero is 25.0% (or 4/16).

d) The vocabulary size with bigram and trigram extraction is 28954, meaning there are 28954 distinct words in the dataset. The number of non-zero features is 1164634. The percent of entries that are nonzero is 1.98% (or 1164634/58892436).

e) The vocabulary size with min_df set to exclude values that show up in fewer than 10 documents is 3064.

f) The vocabulary for the training set is 26879, while the vocabulary for t

(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [15]:
def P3():
### STUDENT START ###
    ''' Initialize over-ambitious answer object 
    '''
    p3 = ProblemAnswer(3)
    
    ''' Run KNN using CV and report f1 score 
        find optimal K value 
    '''
    # Create KNN and CV object
    knn, cv = KNeighborsClassifier(), CountVectorizer()
    
    # Fit CountVectorizer with training and dev datasets
    X = cv.fit_transform(train_data)
    X_dev = cv.transform(dev_data)
    
    # Fit KNN with train data 
    knn.fit(X, train_labels)
    
    # Attempt GridSearchCV to find optimal k
    gs = GridSearchCV(knn, {'n_neighbors':[1,2,3,4,5,6,7,8]})
    gs.fit(X, train_labels)
    best_k = gs.best_params_['n_neighbors']  
    
    # Run KNN with optimal k value 
    knn = KNeighborsClassifier(n_neighbors = best_k)
    knn.fit(X, train_labels)
    
    # Predict with dev data
    predictions = knn.predict(X_dev)
    
    # Generate F1 report
    f1_score_knn = metrics.f1_score(predictions, dev_labels)
    print 'For KNN,\nBest K value: {0}\nF1 Score: {1}\n'.format(
        best_k, f1_score_knn
    )
    
    ''' Create a multinomial NB model
        find optimal value for alpha
    '''
    # Create NB object
    mnb = MultinomialNB()
    
    # Attempt GridSearchCV to find optimal alpha
    gs = GridSearchCV(mnb, {'alpha':[0.0001,0.001,0.01,0.1,1,10,100,1000,10000]})
    gs.fit(X, train_labels)
    best_alpha = gs.best_params_['alpha']
    
    # Run NB model with optimal alpha value
    mnb = MultinomialNB(alpha=best_alpha)
    mnb.fit(X, train_labels)
    
    # Predict with dev data
    predictions = mnb.predict(X_dev)
    
    # Generate F1 report
    f1_score_mnb = metrics.f1_score(predictions, dev_labels)
    print 'For Multinomial NB,\nBest Alpha value: {0}\nF1 Score: {1}\n'.format(
        best_alpha, f1_score_mnb
    )
    
    ''' Create a logistic regression model
        find optimal regularization strength    
    '''
    # Create LR model
    lr = LogisticRegression(penalty='l2')
    
    # Attempt GridSearchCV to find optimal C value
    c_vals = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
    gs = GridSearchCV(lr, {'C':c_vals})
    gs.fit(X, train_labels)
    best_c = gs.best_params_['C']
    
    # Run LR model with optimal C value
    lr = LogisticRegression(penalty='l2',C=best_c)
    lr.fit(X, train_labels)
    
    # Predict with dev data
    predictions = lr.predict(X_dev)
    
    # Generate F1 report
    f1_score_lr = metrics.f1_score(predictions, dev_labels)
    print 'For Logistic Regression,\nBest C value: {0}\nF1 Score: {1}\n'.format(
        best_c, f1_score_lr
    )
    
    p3.add('a','''
        Nearest neighbors likely doesn't work well in this problem 
        because with a sparse matrix consisting of many features, 
        each set of features likely to be much different than the next, 
        there will not be many values that are actually similar to each 
        other by any distance measurement, so the distance measure will
        choose another observation that doesn't appropriately 
        generalize to the test observation. 
    ''')
    
    p3.add('b','''
        Logistic Regression likely performs worse than naive bayes  
        because each feature may rarely occur, and so the maximum 
        likelyhood calculation will have a difficult time evaluating 
        a set of all of the potential features - there will be coefficents 
        for hundreds of features, many of which will only show up once or twice, 
        and so many of them won't be updated from initial values during 
        maximum likelyhood iterations. For Naive Bayes, there will be a probability 
        associated with each feature but the combination of probability for 
        all features in the document produces a prediction, which fits this 
        problem space a bit better than logistic regression.
    ''')
    
    
    def print_coeff_sqsum(C):
        ''' Print sum of squared coefficients for each class '''
        lr = LogisticRegression(penalty='l2',C=C)
        lr.fit(X, train_labels)
        print 'C:',C,np.sum(lr.coef_**2,axis=1)
    
    [ print_coeff_sqsum(c) for c in c_vals ]

    p3.add('c','''
        As C increases (i.e. as regularization decreases), the squared sum 
        of coefficients for the logistic regression model increases. This essentially 
        means that decreasing regularization increases the coefficient "strengths" 
        overall, which is an indication that high C values would probably lead to 
        overfitting. 
    ''')
    
    print '\n',p3

### STUDENT END ###
P3()

For KNN,
Best K value: 1
F1 Score: 0.385769187496

For Multinomial NB,
Best Alpha value: 0.01
F1 Score: 0.784005275779

For Logistic Regression,
Best C value: 0.2
F1 Score: 0.72016213356

C: 0.01 [ 2.54146394  2.93970937  2.86246599  2.2500287 ]
C: 0.05 [ 14.0734849   13.66877727  14.60431154  11.86064108]
C: 0.1 [ 27.12604824  24.66285943  27.4595507   23.02394434]
C: 0.2 [ 49.74967458  42.74064504  49.32426872  42.6702114 ]
C: 0.3 [ 69.27702155  57.88697936  67.90716202  59.77735917]
C: 0.4 [ 86.73631242  71.1126225   84.26173608  75.05021188]
C: 0.5 [ 102.59743971   83.09094079   99.0172645    89.00720041]
C: 0.6 [ 117.23367338   94.07966549  112.52611041  101.84335215]
C: 0.7 [ 130.93432795  104.17726122  125.01795565  113.83319175]
C: 0.8 [ 143.6068096   113.56397142  136.65340897  125.06083457]
C: 0.9 [ 155.58375704  122.47507891  147.62872134  135.58274496]
C: 0.99 [ 165.84633886  130.0837659   156.97985581  144.76852312]

a) Nearest neightbors likely doesn't work well in this p

ANSWER: (see (a),(b), and (c) above)


(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [97]:
def P4():
### STUDENT START ###
    ''' Initialize over-ambitious answer object 
    '''
    p4 = ProblemAnswer(1)
    
    ''' Create LR and CV object
    ''' 
    lr, cv = LogisticRegression(penalty='l2',C=0.2), \
        CountVectorizer(min_df=10)
    X = cv.fit_transform(train_data)
    lr.fit(X, train_labels)
    _names = cv.get_feature_names()
    
    ''' Create LR model with bigram feature set
    '''
    lr2, cv2 = LogisticRegression(penalty='l2',C=0.2), \
        CountVectorizer(analyzer='char_wb',ngram_range = (2,2), \
                        min_df=10)
    Z = cv2.fit_transform(train_data)
    lr2.fit(Z, train_labels)
    _bgnames = cv2.get_feature_names()
    
    def tab_results(lrmodel,names,headers=False):
        ''' Get top five for each category and 
            tabulate reasults using tabulate library
        '''
        from tabulate import tabulate
        
        # Gather coefficients from LR model 
        coeffs = lrmodel.coef_
        
        # Create feature set 
        allfeatures = np.zeros((20))
        for i,x in enumerate(coeffs):
            _topfive = x.argsort()[-5:]
            _topfive_names = [ names[t] for t in _topfive ]
            allfeatures[i*5:i*5+5] = _topfive
            
        # Prepare output table 
        output = []
        for i,x in enumerate(coeffs):
            for e,f in enumerate(allfeatures):
                _coef = coeffs[i,int(f)]
                if len(output)>=len(allfeatures):
                    output[e].extend([_coef])
                else:
                    output.append([names[int(f)],_coef])
                    
        # Print output tabulation
        if headers:
            print tabulate(output,headers),'\n'
        else:
            print tabulate(output),'\n'
    
    ''' Tabulate results for both LR models (regular and bigram)
    '''
    tab_results(lr,_names,newsgroups_train.target_names)
    tab_results(lr2,_bgnames,newsgroups_train.target_names)
    
    p4.add('a','''
        Surprising features in the first table are 'fbi' which has 
        a high coefficient for the category of talk.religion.misc. 
        It seems strange for FBI to be related to religion discussions. 
        Another surprising feature in the table is the word 'bobby' for 
        alt.atheism; however, a quick Google search revealed that 
        Bobby Jindal is a popular discussion topic with Atheists, so 
        this makes sense. For the bigram table, the results are harder to 
        interpret, but for comp.graphics, 'cs' (end of 'graphics') and '3d' 
        make a lot of sense. For talk.religion.misc it is surprising to 
        see that a semicolon '; ' is a popular bigram. 
    
    ''')
    print p4
    
### STUDENT END ###
P4()

              alt.atheism    comp.graphics    sci.space    talk.religion.misc
----------  -------------  ---------------  -----------  --------------------
islam           0.546741        -0.104977     -0.23322             -0.221057
atheists        0.637794        -0.090831     -0.223507            -0.488316
religion        0.657826        -0.444292     -0.584374            -0.0151879
atheism         0.685726        -0.294448     -0.291192            -0.380154
bobby           0.741024        -0.163733     -0.261262            -0.370557
3d             -0.266735         0.798962     -0.479865            -0.27561
computer       -0.0414037        0.842155     -0.531269            -0.346157
file           -0.239271         0.849279     -0.580924            -0.397825
image          -0.394117         0.887072     -0.515662            -0.300102
graphics       -0.539643         1.33387      -0.919498            -0.508001
spacecraft     -0.261324        -0.273495      0.574027            -0.2094

ANSWER: (see tables and answer (a) above)

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [131]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    ''' Preprocess raw text prior to tokenizing 
    '''
    # Lowercase everything
    s = s.lower()
    
    # Replace numbers with number_token
    s = re.sub(r'[0-9]+','number_token',s)
    
    # Replace non-characters with nothing
    s = re.sub(r'[^a-z\s]','',s)
    
    # Shorten long words 
    s = re.sub(r'\w{6,}',lambda x: x.group(0)[:6],s)
    
    return s

### STUDENT END ###

def P5():
### STUDENT START ###
    ''' Initialize over-ambitious answer object 
    '''
    p5 = ProblemAnswer(1)
    
    ''' Create countvectorizer using preprocessor
    '''
    cv1 = CountVectorizer()
    cv2 = CountVectorizer(preprocessor=better_preprocessor)
    
    ''' Transform data for LR model
    '''
    X = cv1.fit_transform(train_data)
    X2 = cv2.fit_transform(train_data)
    X_dev = cv1.transform(dev_data)
    X2_dev = cv2.transform(dev_data)
    _first_vocab_size = X.shape[1]
    _new_vocab_size = X2.shape[1]
    
    ''' Test LR model using preprocessed data
    '''
    lr = LogisticRegression(penalty='l2',C=0.2)
    lr.fit(X, train_labels)
    lr2 = LogisticRegression(penalty='l2',C=0.2)
    lr2.fit(X2, train_labels)
    
    ''' Predict with dev data
    '''
    predictions = lr.predict(X_dev)
    predictions2 = lr2.predict(X2_dev)
    
    ''' Generate F1 reports
    '''
    _f1_score = metrics.f1_score(predictions, dev_labels)
    _f1_score2 = metrics.f1_score(predictions2, dev_labels)
    
    p5.add('a','''
        The preprocessing step decreased the vocab size from {0} 
        to {1}. The F1 score improved from {2} to {3}.
    '''.format(
            _first_vocab_size
            ,_new_vocab_size
            ,round(_f1_score,3)
            ,round(_f1_score2,3)
        ))
    print p5
    
    
### STUDENT END ###

P5()

[u'aa', u'aanta', u'aaske', u'ab', u'abber', u'abdul', u'abili', u'abl', u'abor', u'abou', u'abov', u'abpnu', u'absen', u'absol', u'absor', u'abstr', u'ac', u'acade', u'acadn', u'accel', u'accen', u'accep', u'acces', u'accid', u'accom', u'accor', u'accou', u'accur', u'accus', u'achie', u'aci', u'ackno', u'acoop', u'acor', u'acord', u'acqui', u'acr', u'acrok', u'acron', u'acros', u'acscp', u'act', u'actio', u'activ', u'actua', u'ad', u'ada', u'adag', u'adam', u'adamp', u'adanu', u'add', u'adde', u'addic', u'addit', u'addre', u'adjus', u'adkin', u'admin', u'admir', u'admis', u'admit', u'adnum', u'adob', u'adol', u'adon', u'adrgz', u'adria', u'adul', u'advan', u'adven', u'adver', u'advis', u'advoc', u'ae', u'aeaye', u'aer', u'aerob', u'aerod', u'aeron', u'aeros', u'afa', u'affai', u'affec', u'affia', u'affir', u'affli', u'afpr', u'afrai', u'afric', u'afrp', u'afrpl', u'afslu', u'afte', u'after', u'ag', u'agai', u'again', u'agen', u'agenc', u'agent', u'aggre', u'agnos', u'agora', u'agre', 

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [15]:
#def P6():
    # Keep this random seed here to make comparison easier.
    #np.random.seed(0)

    ### STUDENT START ###
    
    ### STUDENT END ###
#P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [16]:
#def P7():
### STUDENT START ###

## STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.